In [1]:
import numpy as np
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import get_error_measures
from src.models import SimpleANN, LeakySNN, SynapticSNN, DoubleLeakySNN
from src.train_model import training_loop

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
np.random.seed(373)
torch.manual_seed(373)

In [4]:
data06 = pd.read_csv("../data/processed/target06.csv")
data06.drop(columns=["weekday"], inplace=True)

In [5]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(),
                              torch.from_numpy(data06.values[:,-1]).float())  # normalization
train_dataset, test_dataset = torch.utils.data.random_split(all_data, (round(0.45 * len(all_data)), round(0.55 * len(all_data))))
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)

### Two layer perceptron

In [6]:
for i in range(10):
    h = np.random.randint(500, 5000)
    lr = np.random.uniform(0.0001, 0.05)
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: 25, learning rate: {lr}')
    model = SimpleANN(num_inputs=8, num_hidden=h, num_outputs=1)
    training_loop(model, train_loader, test_loader, device, num_epochs=25, lr=lr)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 3261, epochs: 25, learning rate: 0.029204444852406235


100%|██████████| 25/25 [00:07<00:00,  3.23it/s]


Accuracy on validation dataset: 46.45%
MAE: 6.857578805786765, MSE:90.31316158244242,
RMSE: 9.503323712388335, IA: 0.8836526974368362, MAPE: 46.17675171341874%

------------- Random set: 1 -----------------
Hidden: 3325, epochs: 25, learning rate: 0.042285555880229156


100%|██████████| 25/25 [00:05<00:00,  4.43it/s]


Accuracy on validation dataset: 48.93%
MAE: 6.490453949040753, MSE:89.36631800250872,
RMSE: 9.453376010849707, IA: 0.8647578526368259, MAPE: 39.25795705052882%

------------- Random set: 2 -----------------
Hidden: 4481, epochs: 25, learning rate: 0.03575570461126757


...

Accuracy on validation dataset: 48.99%
MAE: 6.599321135807157, MSE:88.17042866836543,
RMSE: 9.389911004283556, IA: 0.8859792466182126, MAPE: 42.48354431089878%

------------- Random set: 8 -----------------
Hidden: 705, epochs: 25, learning rate: 0.035423232586106836


100%|██████████| 25/25 [00:06<00:00,  4.05it/s]


Accuracy on validation dataset: 44.35%
MAE: 6.8390520963913355, MSE:97.77838727917398,
RMSE: 9.888295468844667, IA: 0.8709834525392175, MAPE: 35.601554708629756%

------------- Random set: 9 -----------------
Hidden: 1245, epochs: 25, learning rate: 0.035992301602121676


100%|██████████| 25/25 [00:06<00:00,  4.06it/s]

Accuracy on validation dataset: 49.5%
MAE: 6.502929793430407, MSE:89.8544091878289,
RMSE: 9.479156565213431, IA: 0.8758165391583693, MAPE: 37.44562842254725%


### SNN with LIF neurons

In [7]:
for i in range(10):
    h = np.random.randint(1000, 5000)
    s = np.random.randint(5, 50)
    lr = np.random.uniform(0.0001, 0.005)
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: 15, steps: {s}, learning rate: {lr}')
    model = LeakySNN(num_inputs=8, num_hidden=h, num_outputs=1, num_steps=s)
    training_loop(model, train_loader, test_loader, device, num_epochs=25, lr=lr)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 2022, epochs: 15, steps: 17, learning rate: 0.0003407251725559686


100%|██████████| 25/25 [00:53<00:00,  2.16s/it]


Accuracy on validation dataset: 48.76%
MAE: 6.5703341645721425, MSE:92.24043576583526,
RMSE: 9.604188449100489, IA: 0.8680097854104956, MAPE: 39.09866865382273%

------------- Random set: 1 -----------------
Hidden: 1494, epochs: 15, steps: 30, learning rate: 0.003560334296624881


100%|██████████| 25/25 [01:31<00:00,  3.67s/it]


Accuracy on validation dataset: 48.82%
MAE: 6.45619585404831, MSE:87.04241612941694,
RMSE: 9.329652519221545, IA: 0.8721941762021718, MAPE: 39.478002714855634%

------------- Random set: 2 -----------------
Hidden: 4987, epochs: 15, steps: 20, learning rate: 0.004336489642689075


...

Accuracy on validation dataset: 45.48%
MAE: 6.8757944472350205, MSE:87.10281597085657,
RMSE: 9.332888940240132, IA: 0.8775207111491208, MAPE: 47.92022834662068%

------------- Random set: 8 -----------------
Hidden: 3203, epochs: 15, steps: 22, learning rate: 0.0009318445654611562


100%|██████████| 25/25 [01:08<00:00,  2.74s/it]


Accuracy on validation dataset: 50.19%
MAE: 6.398154672878838, MSE:86.38963856996543,
RMSE: 9.294602657992725, IA: 0.8822516322974007, MAPE: 38.006961702647175%

------------- Random set: 9 -----------------
Hidden: 3187, epochs: 15, steps: 27, learning rate: 0.0016821732566232408


100%|██████████| 25/25 [01:26<00:00,  3.45s/it]


Accuracy on validation dataset: 50.080000000000005%
MAE: 6.319640576213085, MSE:86.20958782191887,
RMSE: 9.284911837056875, IA: 0.8805566451340991, MAPE: 35.23098265476222%


### SNN with Synaptic neurons

In [8]:
for i in range(10):
    h = np.random.randint(1000, 5000)
    s = np.random.randint(5, 50)
    lr = np.random.uniform(0.0001, 0.005)
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: {25}, steps: {s}, learning rate: {lr}')
    model = SynapticSNN(num_inputs=8, num_hidden=h, num_outputs=1, num_steps=s)
    training_loop(model, train_loader, test_loader, device, num_epochs=25, lr=lr)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 3990, epochs: 25, steps: 34, learning rate: 0.004321731145543745


100%|██████████| 25/25 [01:52<00:00,  4.51s/it]


Accuracy on validation dataset: 47.88%
MAE: 6.436269430464784, MSE:81.53674208139402,
RMSE: 9.029769769013717, IA: 0.8814207570983518, MAPE: 42.664463271030485%

------------- Random set: 1 -----------------
Hidden: 3286, epochs: 25, steps: 28, learning rate: 0.0021203096299948045


100%|██████████| 25/25 [01:46<00:00,  4.26s/it]


Accuracy on validation dataset: 50.5%
MAE: 6.254907523598577, MSE:77.31011459206483,
RMSE: 8.792617050233954, IA: 0.894030969437019, MAPE: 41.28562528150957%

------------- Random set: 2 -----------------
Hidden: 2970, epochs: 25, steps: 42, learning rate: 0.0025228026949737606


...

Accuracy on validation dataset: 50.0%
MAE: 6.273682219133301, MSE:77.7858288699951,
RMSE: 8.81962747909429, IA: 0.8982842278414962, MAPE: 41.82061789475755%

------------- Random set: 8 -----------------
Hidden: 4036, epochs: 25, steps: 8, learning rate: 0.004627561752727213


100%|██████████| 25/25 [00:33<00:00,  1.33s/it]


Accuracy on validation dataset: 51.05%
MAE: 6.252818424945221, MSE:81.11350226991209,
RMSE: 9.006303474229151, IA: 0.8926382483770388, MAPE: 38.87455395846558%

------------- Random set: 9 -----------------
Hidden: 3151, epochs: 25, steps: 47, learning rate: 0.0049450017121715805


100%|██████████| 25/25 [02:30<00:00,  6.01s/it]


Accuracy on validation dataset: 49.64%
MAE: 6.311725486672792, MSE:83.89082311134416,
RMSE: 9.159193365757934, IA: 0.8830409907299861, MAPE: 37.133222207019486%


### SNN with additional layer

In [9]:
for i in range(5):
    h = np.random.randint(1000, 5000)
    s = np.random.randint(5, 50)
    lr = np.random.uniform(0.0001, 0.005)
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: 25, steps: {s}, learning rate: {lr}')
    model = DoubleLeakySNN(num_inputs=8, num_hidden=h, num_outputs=1, num_steps=s)
    training_loop(model, train_loader, test_loader, device, num_epochs=25, lr=lr)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 4675, epochs: 25, steps: 34, learning rate: 0.004379094686416036


100%|██████████| 25/25 [03:30<00:00,  8.40s/it]


Accuracy on validation dataset: 47.14%
MAE: 6.617967626786924, MSE:84.66331884069949,
RMSE: 9.201267241021721, IA: 0.8802051887766367, MAPE: 44.196257041855716%

------------- Random set: 1 -----------------
Hidden: 3554, epochs: 25, steps: 12, learning rate: 0.003589747747398896


100%|██████████| 25/25 [01:17<00:00,  3.09s/it]


Accuracy on validation dataset: 48.209999999999994%
MAE: 6.529579632695155, MSE:88.38105925045531,
RMSE: 9.401120106160505, IA: 0.8871405650229348, MAPE: 36.56310142109275%

------------- Random set: 2 -----------------
Hidden: 4453, epochs: 25, steps: 15, learning rate: 0.0041471944508329785


...

Accuracy on validation dataset: 48.209999999999994%
MAE: 6.540765011746526, MSE:84.58752544629766,
RMSE: 9.197147679922164, IA: 0.8832320264916831, MAPE: 42.79645478688669%

------------- Random set: 3 -----------------
Hidden: 1948, epochs: 25, steps: 22, learning rate: 0.000626213398548346


100%|██████████| 25/25 [02:11<00:00,  5.24s/it]


Accuracy on validation dataset: 48.26%
MAE: 6.680241693263797, MSE:89.24091784012374,
RMSE: 9.446741122743003, IA: 0.8764734349975523, MAPE: 43.833101773962795%

------------- Random set: 4 -----------------
Hidden: 4357, epochs: 25, steps: 28, learning rate: 0.0048408839633424525


100%|██████████| 25/25 [02:58<00:00,  7.16s/it]


Accuracy on validation dataset: 43.65%
MAE: 6.842782654573248, MSE:95.6660819464583,
RMSE: 9.780903943218044, IA: 0.877891360592146, MAPE: 35.82834764783045%
